<a href="https://colab.research.google.com/github/OryzaPhytoE/Test/blob/main/PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [4]:
# Random seed for consistency
np.random.seed(42)

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Label Encoder use to Encode target labels with value between 0 and n_classes-1
Encoder = LabelEncoder()

# TfidfVectorizer Convert a collection of raw documents to a matrix of TF-IDF features.
Tfidf_vect = TfidfVectorizer()

In [16]:
from google.colab import drive
drive.mount('/content/drive')

data_train = pd.read_csv('/content/drive/My Drive/PI/Data_latih.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Bentuk data_train

data_train.head()

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [18]:
# Jumlah text per-kategori(label)

data_train['label'].value_counts()

,count
label,
1,3465
0,766


In [22]:
# Dari cell sebelumnya terlihat jelas bahwa dataset kita sangat tidak balance
# Untuk membuat datasetnya balance

# Pilih dataset dengan label 1 dan lakukan randomisasi untuk setiap baris datanya
false_news = data_train[data_train['label'] == 1].sample(frac=1)

# Concat dataset berlabel 1 yang telah dipilih dengan dataset berlabel 0
# dimana jumlah dataset berlabel 1 yang digabungkan sejumlah banyak dataset berlabel 0 + 200
true_fact = data_train[data_train['label'] == 0]
df = pd.concat([true_fact, false_news[:len(true_fact) + 200]]) # Use pd.concat instead of append

df

,ID,label,tanggal,judul,narasi,nama file gambar
19,6775,0,25-Oct-17,Sandi Sarankan Warga Ngadu ke Kelurahan Saja,Astaga! Sandiaga Sarankan Warga Ngadu ke Kelur...,6775.jpg
21,7038,0,13-Jul-17,"Irina, Istri Ahli IT Hermansyah disebut Mantan...","Irina, Istri Ahli IT Hermansyah disebut Mantan...",7038.jpg
22,7214,0,8-Mar-17,"Nezar Patria adalah Kader PKI, Ikut Rapat Seti...","Nezar Patria adalah Kader PKI, Ikut Rapat Seti...",7214.jpg
55,13483,0,17-Jul-18,Coach Indra Sjafri: Saya Pernah 1 Tahun Tak Di...,Kendati sukses mengantarkan Timnas Indonesia U...,13483.png
58,13907,0,28-Apr-18,Kronologis Polisi Berondong Sedan Terobos Razi...,"Innalillah… Hanya Karena Lalai Terobos Razia, ...",13907.png
...,...,...,...,...,...,...
3288,773600,1,22-Apr-19,Pesan Dari Jusuf Kalla,PESAN DARI JUSUF KALLA,773600.jpg
1320,305799,1,25-Sep-19,Polisi Pastikan Tak Ada Korban Meninggal dalam...,Tidak ada korban yang meninggal. Kabar meningg...,305799.png
2116,496484,1,5-Sep-18,@RatnaSpaet: Masih mau 2 Periode?,Masih mau 2 Periode? Pakai akal pikiranmu – Gu...,496484.jpg
2241,523406,1,18-Feb-17,AOCT,Sebarkan ini lah yg mengheck dan mengganti dat...,523406.jpg


In [27]:
# Kita akan menggunakan fitur narasi saja dalam melakukan prediksi terhadap label
feature = df['narasi']
label = df['label']

In [28]:
# Mengubah semua huruf pada setiap baris menjadi huruf kecil dan
# melakukan stemming pada setiap baris
lower = [stemmer.stem(row.lower()) for row in feature]

# Hasil stem dan lower
lower[:5]

['astaga sandiaga saran warga ngadu ke lurah saja tak perlu ke balai kota',
 'irina istri ahli it hermansyah sebut mantan psk',
 'nezar patria adalah kader pki ikut rapat tiap malam di istana telah jam 8 malam demikian tuduh dari alfian tandjung',
 'kendati sukses antar timnas indonesia u-19 juara piala aff u-19 2013 dan lolos ke putar final piala asia u-19 2014',
 'innalillah hanya karena lalai terobos razia mobil isi satu keluarga tembak polisi satu orang tewas']

In [32]:
# Melakukan tokenisasi untuk setiap baris dataset
import nltk

nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [33]:
# Melakukan tokenisasi untuk setiap baris dataset
tokens = [word_tokenize(element) for element in lower]

# Hasil tokenisasi setiap baris
tokens[:1]

[['astaga',
  'sandiaga',
  'saran',
  'warga',
  'ngadu',
  'ke',
  'lurah',
  'saja',
  'tak',
  'perlu',
  'ke',
  'balai',
  'kota']]

In [34]:
# train_test_split digunakan untuk memecah dataset menjadi 2 bagian
# X_train dan y_train mewakili data yang akan dilakukan pada fitting model(Training model)
# X_test dan y_test  mewakili data yang akan dilakukan pada evaluasi model
X_train, X_test, y_train, y_test = train_test_split(tokens, label, test_size=0.3, stratify=label)

In [35]:
# Melihat ukuran data latih dan data uji
print('X_train : ', len(X_train))
print('X_test : ', len(X_test))

X_train :  1212
X_test :  520


In [36]:
# Encoder for Data Features
df = pd.DataFrame(X_test)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,hebat,ah,formulir,lapor,lahir,sudah,tidak,guna,kolom,agama,...,None,None,None,None,None,None,None,None,None,None
1,artikel,sebut,klaim,pbb,bantu,bashar,assad,serang,rumah,sakit,...,None,None,None,None,None,None,None,None,None,None
2,15,hari,sembuh,menhub,budi,karya,kembali,nyata,positif,covid-19,...,None,None,None,None,None,None,None,None,None,None
3,hari,ini,pasang,baliho,contoh,surat,suara,pilpres,oleh,kpu,...,None,None,None,None,None,None,None,None,None,None
4,sejak,minggu,8,oktober,2017,lalu,masyarakat,kota,balikpapan,buat,...,None,None,None,None,None,None,None,None,None,None


In [37]:
# Encoder for Data Label
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

y_train

array([1, 0, 0, ..., 0, 1, 1])

In [38]:
# Fitting dataset terhadap tf-idf
Tfidf_vect.fit(["".join(row) for row in X_train])

TfidfVectorizer()

In [39]:
# Mentransformasikan hasil fitting terhadap data X_train dan X_test
X_train_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_train])
X_test_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_test])

In [40]:
# Classifier - Algorithm - SVM
# fitting/training datasets pada algoritma SVM(Support Vector Machine)
SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma="auto", verbose=True)
SVM.fit(X_train_Tfidf, y_train)  # predict the labels on validation dataset

# Menggunakan metrics accuracy untuk melihat performa model
predictions_SVM = SVM.predict(X_test_Tfidf)
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100)

[LibSVM]SVM Accuracy Score ->  55.96153846153846


In [41]:
rf = RandomForestClassifier()
rf.fit(X_train_Tfidf, y_train)

prediction_rf = rf.predict(X_test_Tfidf)
print("RandomForest Accuracy Score -> ", accuracy_score(prediction_rf, y_test)*100)

RandomForest Accuracy Score ->  56.15384615384615


In [42]:
# Classifier - Algorithm - GAN
# GRN: Arsitektur GAN yang dikombinasikan dengan fungsi Round pada elemen discrimator untuk mengurangi redundancy features pada data yang besar
#Input : X_train, X_test, y_train, y_test
y_train

array([1, 0, 0, ..., 0, 1, 1])